In [1]:
import psycopg2
import csv
import glob
import numpy as np 

In [2]:
conn = psycopg2.connect("dbname=mod2project")
cur = conn.cursor()

In [3]:
create_query_1 = """
CREATE TABLE courses(
  uuid text PRIMARY KEY
, name text 
, number integer
)
"""

In [4]:
create_query_2 = """
CREATE TABLE course_offerings(
  uuid text
, course_uuid text 
, term_code integer
, name text
)
"""

In [5]:
create_query_3 = """
CREATE TABLE grade_distributions(
  course_offering_uuid text
, section_number text 
, a_count integer
, ab_count integer
, b_count integer
, bc_count integer
, c_count integer
, d_count integer
, f_count integer
, s_count integer
, u_count integer
, cr_count integer
, n_count integer
, p_count integer
, i_count integer
, nw_count integer
, nr_count integer
, other_count integer
, PRIMARY KEY (course_offering_uuid, section_number)
)
"""

In [6]:
create_query_4 = """
CREATE TABLE instructors(
  id integer PRIMARY KEY
, name text
)
"""

In [7]:
create_query_5 = """
CREATE TABLE rooms(
  uuid text PRIMARY KEY
, facility_code text
, room_code text
)
"""

In [8]:
create_query_6 ="""
CREATE TABLE schedules(
  uuid text PRIMARY KEY
, start_time integer
, mon integer
, tues integer
, wed integer
, thurs integer
, fri integer
, sat integer
, sun integer
)
"""

In [9]:
create_query_7 = """
CREATE TABLE sections(
            uuid text PRIMARY KEY,
            course_offering_uuid text,
            section_type text,
            number text,
            room_uuid text,
            schedule_uuid text)"""

In [10]:
create_query_8 = """
CREATE TABLE subject_memberships(
            subject_code integer PRIMARY KEY,
            course_offering_uuid text)"""

In [11]:
create_query_9 = """
CREATE TABLE subjects(
            code integer PRIMARY KEY,
            name text,
            abbreviation text)"""

In [12]:
create_query_10 = """
CREATE TABLE teachings(
            instructor_id integer PRIMARY KEY,
            section_uuid text)"""

In [13]:
csvs_for_table = glob.glob('raw_data/*.csv')

In [14]:
queries_list = [create_query_1,create_query_2,create_query_3,create_query_4
               ,create_query_5,create_query_6,create_query_7
               ,create_query_8,create_query_9,create_query_10]

In [15]:
table_names = [x.split('/')[1][:-4] for x in csvs_for_table]

In [16]:
for q_num, table_name in enumerate(table_names):
    print(table_name)
    print(q_num)
    table = table_name
    fields = tuple(line.strip(", ").split()[0] for 
                   line in (queries_list[q_num]).split('\n')[2:-3])
    print(fields)

course_offerings
0
('uuid', 'name')
courses
1
('uuid', 'course_uuid', 'term_code')
grade_distributions
2
('course_offering_uuid', 'section_number', 'a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count', 's_count', 'u_count', 'cr_count', 'n_count', 'p_count', 'i_count', 'nw_count', 'nr_count', 'other_count')
instructors
3
('id',)
rooms
4
('uuid', 'facility_code')
schedules
5
('uuid', 'start_time', 'mon', 'tues', 'wed', 'thurs', 'fri', 'sat')
sections
6
('uuid', 'course_offering_uuid', 'section_type')
subject_memberships
7
()
subjects
8
()
teachings
9
()


In [ ]:
query = f"""
INSERT INTO {table}
  ({', '.join(fields)})
VALUES
  ({', '.join('%s' for _ in range(len(fields)))})
"""

In [26]:
cur.execute(create_query)
conn.commit()

NameError: name 'table' is not defined

In [ ]:
with open('courses.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row.
    for row in reader:
        cur.execute(
            query,
            row
        )
conn.commit()